In [95]:
import sys
import os
# Add the src directory to Python path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import the modules
import clathrate_analysis
import truncation_analysis
import tomogram_utils

# Reload the modules
import importlib
importlib.reload(clathrate_analysis)
importlib.reload(truncation_analysis)
importlib.reload(tomogram_utils)

# Now import the package
from clathrate_analysis import *#parse_particles_and_shape, plot_clathrate_cavities,  get_truncated_bipyramid_geometry,
from truncation_analysis import *#plot_truncation_cavity_comparison
from tomogram_utils import *#create_tomogram_from_particles

In [113]:
# Load the file
# filename = 'C:\\Users\\b304014\\Software\\blee\\models\\Cages\\Cage A new.pos'
filename = 'C:\\Users\\b304014\\Software\\blee\\models\\ClaS_bipyramid_averaged.pos'

# Parse particles and shape data
particles, shape_vertices, shape_color, simulation_data = parse_particles_and_shape(filename)

# Set truncation factor
truncation_factor = 0.4  # Can be adjusted between 0.0 and 1.0

# Improved cavity detection parameters
grid_size = 128  # Increased from 64 for better resolution
min_cavity_size = 4000  # Reduced to detect smaller cavities
padding = 0.15  # Moderate padding for better cavity detection

# # For 2x2x2 duplication
# grid_size = 128*2
# min_cavity_size = 100 # Number of voxels for a cavity to be considered

print(f"Minimum cavity size: {min_cavity_size}")
print(f"Grid size: {grid_size}")
print(f"Truncation factor: {truncation_factor}")
print(f"Padding: {padding}")

# Get truncated shape faces for tomogram creation
_, shape_faces, _ = get_truncated_bipyramid_geometry(
    shape_vertices, 
    shape_color,
    truncation_factor=truncation_factor
)

# Create nxnxn duplicated particles
duplicated_particles = duplicate_unit_cell(particles, nx=2, ny=2, nz=2, simulation_data=simulation_data)
duplicated_positions = [(pos, quat) for pos, quat, _ in duplicated_particles]


Found shape definition: 5 vertices, color: rgba(255,0,0,1.0)
Found 92 particles

Simulation Metadata:
  Date: Friday, July 7, 2023, 3:08:51 PM
  Steps: 65400
  Time: 24.695584
  Volume: 9.03524271
  Packing Fraction: 0.599999998
  Box Dimensions: 2.083 × 2.083 × 2.083
Minimum cavity size: 4000
Grid size: 128
Truncation factor: 0.4
Padding: 0.15


In [114]:
# # Plot the duplicated truncated particles with cavities
# print(f"\nPlotting duplicated particles with truncation factor {truncation_factor} and cavities...")
# cavity_centers, cavity_volumes, cavity_radii = plot_clathrate_cavities(
#     particles=duplicated_positions,
#     shape_vertices=shape_vertices, 
#     shape_color=shape_color,
#     geometry_func=get_truncated_bipyramid_geometry,
#     truncation_factor=truncation_factor,
#     show_particles=True,
#     show_cavities=True,
#     show_spheres=True,
#     grid_size=grid_size,
#     padding=padding,  # Use the improved padding value
#     simulation_data=simulation_data,
#     min_cavity_size=min_cavity_size,
#     keep_largest_cavity_only=False  # Show all cavities first
# )

In [123]:
cavities = detect_simple_cavities(
    particles=duplicated_positions, 
    shape_vertices=shape_vertices, 
    shape_color=shape_color,
    grid_size=128, 
    padding=0.2,
    min_radius=0.16,#0.15,#0.2,#0.33, 
    min_separation=0.2,  
    geometry_func=get_truncated_bipyramid_geometry, 
    truncation_factor=truncation_factor,
    min_surrounding_particles=2,
    max_empty_neighbors_fraction=0.8,
    boundary_margin=0.15
)


Simplified Cavity Detection:

Parameters:
  min_radius: 0.16
  min_separation: 0.2
  boundary_margin: 0.15
  min_surrounding_particles: 2
  max_empty_neighbors_fraction: 0.8

1. Creating particle grid...
Structure boundaries:
  Min: [-1.16675998 -1.06939813 -1.10797323]
  Max: [3.13704516 3.10623709 3.07985328]
Box size: [4.30380514 4.17563522 4.18782651]
Grid resolution: 0.0471 units per voxel

2. Filling particle voxels...
Filled voxels: 429490

3. Finding cavities...
Initial empty voxels: 1667662
Valid region voxels (after boundary filtering): 606816
Remaining cavity voxels (after boundary filtering): 261163
Maximum distance to particle: 0.1997 real units

Found 128 potential cavity centers after initial filtering
  Min radius threshold: 0.16
  Neighborhood size for local maxima: 4 voxels

4. Calculating cavity properties...

Detailed cavity analysis:

Analyzing potential cavity at [-0.49765277  0.6758869  -0.22787532]
Radius: 0.1997 units (4 voxels)
  Shell analysis:
    Empty fra

In [124]:
# Extract centers, radii and volumes into separate lists
cavity_centers = [cavity['center'] for cavity in cavities]
cavity_radii = [cavity['radius'] for cavity in cavities]
cavity_volumes = [cavity['volume'] for cavity in cavities]

print("Cavity centers:", cavity_centers)
print("Cavity radii:", cavity_radii) 
print("Cavity volumes:", cavity_volumes)

# After detecting cavities with detect_clathrate_cavities or find_central_cavity:
plot_cavity_cubes(
    particles=duplicated_positions,
    cavity_centers=cavity_centers,
    cavity_radii=cavity_radii,
    shape_vertices=shape_vertices,
    shape_color=shape_color,
    geometry_func=get_truncated_bipyramid_geometry, 
    show_particles=True,  # Set to False to hide original particles
    cube_opacity=0.3,     # Adjust for better visibility
    simulation_data=simulation_data,
    truncation_factor=truncation_factor
)

Cavity centers: [array([-0.49765277,  0.6758869 , -0.22787532]), array([-0.49765277,  0.6758869 ,  1.83332055]), array([ 1.57355345,  0.6758869 , -0.22787532]), array([-0.49765277, -0.37454633, -0.22787532]), array([-0.49765277,  1.72632014, -0.22787532]), array([-0.49765277,  1.72632014,  1.83332055]), array([1.57355345, 0.6758869 , 1.83332055]), array([ 1.57355345,  1.72632014, -0.22787532]), array([-0.87423572,  0.6758869 , -0.18207096]), array([-0.87423572,  0.6758869 ,  1.8791249 ]), array([-0.45057991,  2.77675337, -0.22787532]), array([-0.40350704,  2.77675337,  1.83332055]), array([ 1.57355345,  2.77675337, -0.31948402]), array([1.57355345, 2.77675337, 1.78751619]), array([-0.87423572,  2.54839832,  2.06234231]), array([0.53795034, 0.17350579, 1.83332055]), array([0.53795034, 2.22870125, 1.83332055]), array([2.65622943, 0.17350579, 1.83332055]), array([ 2.65622943,  2.22870125, -0.22787532]), array([2.65622943, 2.22870125, 1.83332055]), array([-0.87423572,  0.44753185,  0.00114

In [126]:
plot_truncated_particle(
    pos=(0, 0, 0),  # center position
    quat=(1, 0, 0, 0),  # no rotation
    shape_vertices=shape_vertices,
    shape_color=shape_color,
    truncation_factor=0.0,  # or 0.0 for non-truncated bipyramid
    plot_type='both',  # 'surface', 'wireframe', or 'both'
    show_axes=True
)